In [1]:
from torch_geometric.datasets import KarateClub
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.utils import to_networkx
from sklearn.manifold import SpectralEmbedding


In [9]:
# concatenate nx.karate_club_graph() and torch_geometric.datasets.KarateClub() to get the same graph



/home/codespace/.python/current/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


TypeError: eq() received an invalid combination of arguments - got (Tensor, csr_array), but expected one of:
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, Number other, *, Tensor out)


In [5]:
edge_list = [(edge[0], edge[1]) for edge in combined_representation]
graph = nx.from_edgelist(edge_list)
X = nx.to_numpy_array(graph)
position = nx.spring_layout(graph, seed=55)
N = len(X)
X

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [6]:
graph = nx.from_edgelist(data.edge_index.t().tolist())
X = nx.to_numpy_array(graph)
position = nx.spring_layout(graph, seed=55)
N = len(X)
X

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [7]:
labels = []
for i in graph.nodes:
    if 'club' in graph.nodes[i]:
        club_names = graph.nodes[i]['club']
        labels.append(1 if club_names == 'Officer' else 0)
    else:
        labels.append(0)  # Default label if 'club' key is not present
Y = np.array(labels)
X = torch.FloatTensor(X)
y = torch.LongTensor(Y)
Y.shape

(34,)

In [ ]:
# Split the dataset into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class KarateClubClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(KarateClubClassifier, self).__init__()
        self.layers = []
        for i, hidden_size in enumerate(hidden_sizes):
            if i == 0:
                self.layers.append(nn.Linear(input_size, hidden_size))
            else:
                self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_size))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.model(x)


In [ ]:
input_size = X.shape[1]
layer_result = []

def layer_run(layer_size):
    hidden_sizes = [64 for i in range(layer_size)] 
    output_size = 2
    model = KarateClubClassifier(input_size, hidden_sizes, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(70):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('layer_size: {} , Accuracy:  {} %'.format(layer_size,100 * correct / total))
    layer_result.append(100 * correct / total)



### Section A&B: 2-20 layers

In [ ]:
for i in range(1, 20):
    layer_run(i)

### Section C:

by increasing the number of layers, the accuracy of the model increases. However, the accuracy of the model is not always increasing. For example, when the number of layers is 7, the accuracy of the model is 100, but when the number of layers is 8, the accuracy of the model is 71. This is because the model is overfitting. for high number of layers, the model is overfitting and the accuracy of the model is decreasing and model going to be unstable.